Extraction and quantification of market sentiment from textual data such as news headlines or social media posts related to stock tickers. It fetches recent text data using AlphaVantage Global News API, preprocesses and cleans the text for analysis, and then applies the VADER sentiment analyser to assign sentiment scores (positive, negaitve, neutral and compound) to each piece of text. 
Scores are then aggregated over chosen time intervals to create a time-aligned sentiment dataset that can be merged with market price data for further modeling and visualisation. 

In [ ]:
# import libraries 
import requests 
import pandas as pd 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

In [6]:
def fetch_alphavantage_news_df(api_url: str) -> pd.DataFrame: 
    """
    Fetches news data from Alphavantage Global News API URL and returns
    a cleaned pandas DataFrame with parsed dates.
    
    Parameters:
        api_url (str): Fully constructed API URL with key and parameters.
    
    Returns:
        pd.DataFrame: DataFrame with news articles.
    """
    response = requests.get(api_url)
    data = response.json()

    # Extract the "feed" list from the JSON response 
    news_list = data.get("feed", [])
    
    # Convert list of news dictionaries into a DataFrame
    df = pd.DataFrame(news_list)

    # Parse the published dates into datetime objects 
    df["time_published"] = pd.to_datetime(df["time_published"], errors = "coerce")

    return df

In [7]:
url = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=1D1C3X346D6ATHPG"

In [11]:
news_df = fetch_alphavantage_news_df(url)

In [12]:
# Visualise the dataframe 
print(news_df.columns.tolist())
print(news_df.head())

['title', 'url', 'time_published', 'authors', 'summary', 'banner_image', 'source', 'category_within_source', 'source_domain', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'ticker_sentiment']
                                               title  \
0  Apple's AI Momentum Is Building -- Here's What...   
1  Microsoft, Apple, Amazon, and Meta Just Gave N...   
2  Bulls And Bears: AMD, Apple, Eli Lilly - And W...   
3  AAPL NEWS: Did Apple Inc. Mislead Investors? C...   
4                     Where Will Apple Be in 1 Year?   

                                                 url      time_published  \
0  https://www.fool.com/investing/2025/08/09/appl... 2025-08-09 17:00:00   
1  https://www.fool.com/investing/2025/08/09/micr... 2025-08-09 13:05:00   
2  https://www.benzinga.com/markets/market-summar... 2025-08-09 12:19:59   
3  https://www.benzinga.com/pressreleases/25/08/g... 2025-08-09 11:36:00   
4  https://www.fool.com/investing/2025/08/09/wher... 2025-08-09 10:35:00 

In [ ]:
# Apply VADER sentiment analysis on the news headlines using the "title" and "summary" columns

sia = SentimentIntensityAnalyzer()
def get_sentiment_scores(text): 
    if isinstance(text, str):
        return sia.polarity_scores(text)
    else:
        return {'neg': None, 'neu': None, 'pos': None, 'compound': None}


In [14]:
# Example usage 
news_df[["neg", "neu", "pos", "compound"]] = news_df["title"].apply(get_sentiment_scores).apply(pd.Series)

In [15]:
news_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,neg,neu,pos,compound
0,Apple's AI Momentum Is Building -- Here's What...,https://www.fool.com/investing/2025/08/09/appl...,2025-08-09 17:00:00,[Stefon Walters],Apple plans to ramp up investments to catch up...,https://media.ycharts.com/charts/5f2f4d9e75818...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.242728,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0622...",0.000,1.000,0.000,0.0000
1,"Microsoft, Apple, Amazon, and Meta Just Gave N...",https://www.fool.com/investing/2025/08/09/micr...,2025-08-09 13:05:00,[Jennifer Saibil],There's a lot of room for optimism as Nvidia i...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.208394,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1708...",0.000,0.608,0.392,0.7003
2,"Bulls And Bears: AMD, Apple, Eli Lilly - And W...",https://www.benzinga.com/markets/market-summar...,2025-08-09 12:19:59,[Benzinga Senior Editor],Benzinga examined the prospects for many inves...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.122102,Neutral,"[{'ticker': 'LLY', 'relevance_score': '0.14751...",0.000,0.949,0.051,0.2500
3,AAPL NEWS: Did Apple Inc. Mislead Investors? C...,https://www.benzinga.com/pressreleases/25/08/g...,2025-08-09 11:36:00,[Globe Newswire],"NEW YORK, Aug. 09, 2025 ( GLOBE NEWSWIRE ) -- ...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,News,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.145500,Neutral,"[{'ticker': 'WBD', 'relevance_score': '0.06869...",0.000,1.000,0.000,0.0000
4,Where Will Apple Be in 1 Year?,https://www.fool.com/investing/2025/08/09/wher...,2025-08-09 10:35:00,[Neil Patel],The leader in consumer electronics has made fo...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.156079,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.7836...",0.000,1.000,0.000,0.0000
5,"The Best Stocks to Invest $50,000 in Right Now",https://www.fool.com/investing/2025/08/09/the-...,2025-08-09 09:45:00,[Keithen Drury],Big tech is still one of the best places to in...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.366556,Bullish,"[{'ticker': 'META', 'relevance_score': '0.1169...",0.000,0.656,0.344,0.6369
6,Apple's New Artificial Intelligence ( AI ) S...,https://www.fool.com/investing/2025/08/09/appl...,2025-08-09 09:30:00,[Keithen Drury],CEO Tim Cook revealed a potential path to beco...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.6...",0.098990,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0586...",0.000,0.795,0.205,0.4767
7,3 No-Brainer Warren Buffett Stocks to Buy Righ...,https://www.fool.com/investing/2025/08/09/3-no...,2025-08-09 08:05:00,[James Brumley],The Oracle of Omaha's approach to picking stoc...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.163681,Somewhat-Bullish,"[{'ticker': 'OXY', 'relevance_score': '0.11270...",0.000,1.000,0.000,0.0000
8,"Forget the ""Magnificent Seven."" It's Time to S...",https://www.fool.com/investing/2025/08/09/forg...,2025-08-09 08:00:00,[Daniel Foelber],The megacap all-star team is long overdue for ...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.240743,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.2813...",0.113,0.655,0.232,0.4588
9,Apple's In F1 Race While Tim Cook Sips